In [1]:
from requests import get
from bs4 import BeautifulSoup as Soup
import pandas as pd
import requests
import numpy as np
print("Libraries Installed")

Libraries Installed


In [2]:
url=get("https://www.goodreads.com/list/show/5")
request=url.text
soup_data=Soup(request,"html.parser")

In [3]:
#Creating URL list as a function
def url_list(soup_data):
    urls=soup_data.findAll(class_="bookTitle")
    url_list=["https://www.goodreads.com"+str(list(str(url).split(" "))[2])[6::] for url in urls]
    return url_list

In [4]:
#Creating Title list as a function
def title_list(soup_data):
    titles=soup_data.findAll(class_="bookTitle")
    title_list=[title.text.strip() for title in titles]
    return title_list

In [5]:
#Creating Author list as a function
def author_list(soup_data):
    authors=soup_data.findAll(class_="authorName")
    author_list=[author.text for author in authors]
    return author_list

In [6]:
#Creating Rating list as a function
def ratings_list(soup_data):
    ratings=soup_data.findAll("div",{"id":"bookMeta"})
    try:
        return ratings[0].find("a",{"class":"gr-hyperlink"}).text.strip().split("\n")[0].replace(",","")
    except:
        return np.nan

In [7]:
#Creating Review list as a function
def reviews_list(soup_data):
    reviews=soup_data.findAll("div",{"id":"bookMeta"})
    try:
        return int(reviews[0].text.strip().split("\n")[-2].strip().replace(",",""))
    except:
        return np.nan

In [8]:
#Creating Avg Rating list as a function
def avg_list(soup_data):
    avg_lists=[]
    avg_ratings=soup_data.findAll(class_="minirating")
    for avg in range(len(avg_ratings)):
        try:
            avg_lists.append(round(float(avg_ratings[avg].text[0:4].strip())))
        except:
            avg_lists.append(np.nan)
    return avg_lists

In [9]:
#Creating Number of Page list as a function
def pages_list(soup_data):
    num_pages=soup_data.findAll("span",{"itemprop":"numberOfPages"})
    try:
        return int(num_pages[0].text.split(" ")[0])
    except:
        return np.nan

In [10]:
#Creating Published Year list as a function
def published_list(soup_data):
    published_year=soup_data.findAll("div",{"id":"details"})
    published_list=[]
    try:
        return published_year[0].text.split("\n")[4][-4::]
    except:
        return np.nan

In [11]:
#Creating Series Boolean list as a function
def series_list(soup_data):
    series=soup_data.findAll("h2",{"id":"bookSeries"})
    
    try:
        if series[0].text.strip()=="":
            return 0
        else:
            return 1
    except:
        return np.nan


In [12]:
#Creating Genres list as a function
def genres_list(soup_data):
    genres=soup_data.findAll(class_="rightContainer")
    genres_lists=[]
    try:
        for i in range(1,10,3):
            genres_lists.append(genres[0].text.strip().split("Genres")[1].split("\n\n\n")[i].split("\n")[-1].strip())
        return genres_lists[0:3]
    except:
        return np.nan


In [13]:
#Creating Awards list as a function
def awards_list(soup_data):
    awards=soup_data.findAll(class_="award")
    try:
        awards_lists=[award.text.strip() for award in awards]
        return awards_lists
    except:
        return np.nan

In [25]:
#Creating Places List as a Function
def places_list(soup_data):
    places=soup_data.findAll(id="bookDataBox")
    try:
        pre_list_places=places[0].text.split("\nSetting")[1].strip().split("Literary Awards")[0].strip().split("\n\n\n")
        places_lists=[places.strip().split("\n\n") for places in pre_list_places ]
        #deleting empty list values
        filter_object = filter(lambda x: x != '', places_lists)
        return list(filter_object)
    except:
        return np.nan
    


In [24]:
#Creating Function soup object as input
def get_data(url):
    url_get=get(url)
    request=url_get.text
    soup_data=Soup(request,"html.parser")
    return soup_data

#Creating Dataframe as a function
def book(url):
    soup_data=get_data(url)
    
    #Creating URL List From Function
    url_lists=url_list(soup_data)

    #Creating Title List From Function
    title_lists=title_list(soup_data)
    
    #Creating Author List From Function
    author_lists=author_list(soup_data)
    
    #Creating Avg List From Function
    avg_lists=avg_list(soup_data)
    
    pages_lists,ratings_lists,reviews_lists,published_lists,series_lists,genres_lists,awards_lists,places_lists=[],[],[],[],[],[],[],[]

    
    #Creating "for loop" for iterating through the pages
    for urls in url_lists:
        soup_data2=get_data(urls)
        
        #Creating Pages List From Function
        pages_lists.append(pages_list(soup_data2))
        
        #Creating Ratings List From Function
        ratings_lists.append(ratings_list(soup_data2))
        
        #Creating Reviews List From Function
        reviews_lists.append(reviews_list(soup_data2))
        
        #Creating Published Year List From Function
        published_lists.append(published_list(soup_data2))
        
        #Creating Series Boolean List From Function
        series_lists.append(series_list(soup_data2))
        
        #Creating Genres List From Function
        genres_lists.append(genres_list(soup_data2))
        
        #Creating Awards List From Function
        awards_lists.append(awards_list(soup_data2))
          
        #Creating Places List From Function
        places_lists.append(places_list(soup_data2))

    df_dict={"URL":url_lists,"Title":title_lists,"Author":author_lists,"Number of Ratings":ratings_lists,"Number of Reviews":reviews_lists,
        "Average Ratings":avg_lists,"Number of Pages":pages_lists,"Published Year":published_lists,
        "Series":series_lists,"Genres":genres_lists,"Awards":awards_lists,"Places":places_lists}
    #df=pd.DataFrame(data=df_dict)
    return df_dict
    #return list(url_lists,title_lists,author_lists,ratings_lists,reviews_lists,avg_lists,pages_lists,published_lists,series_lists,genres_lists,awards_lists,places_lists)


url="https://www.goodreads.com/list/show/5"
book(url)
    
    

{'URL': ['https://www.goodreads.com/book/show/136251.Harry_Potter_and_the_Deathly_Hallows"',
  'https://www.goodreads.com/book/show/2767052-the-hunger-games"',
  'https://www.goodreads.com/book/show/1.Harry_Potter_and_the_Half_Blood_Prince"',
  'https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire"',
  'https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix"',
  'https://www.goodreads.com/book/show/19063.The_Book_Thief"',
  'https://www.goodreads.com/book/show/18619684-the-time-traveler-s-wife"',
  'https://www.goodreads.com/book/show/77203.The_Kite_Runner"',
  'https://www.goodreads.com/book/show/4667024-the-help"',
  'https://www.goodreads.com/book/show/128029.A_Thousand_Splendid_Suns"',
  'https://www.goodreads.com/book/show/6148028-catching-fire"',
  'https://www.goodreads.com/book/show/41865.Twilight"',
  'https://www.goodreads.com/book/show/43641.Water_for_Elephants"',
  'https://www.goodreads.com/book/show/2429135.The_Girl_with_the_D

In [ ]:
#df.to_csv(r'C:\Users\aktum\Desktop\First.csv', index = False)


In [27]:
#Creating list page for 10 pages of books
main_page="https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page="
list_pages={main_page+str(page) for page in range(1,11)}
list_pages

{'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=1',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=10',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=2',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=3',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=4',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=5',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=6',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=7',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=8',
 'https://www.goodreads.com/list/show/5.Best_Books_of_the_Decade_2000s?page=9'}

In [ ]:
#Creating "For Loop" for 10 pages of books
d=book(url)


In [ ]:
for url in list_pages:
    page_lists=book(url)
    for column in page_lists:
        column.append(column)
page_lists
    
    

In [22]:
d+d

,URL,Title,Author,Number of Ratings,Number of Reviews,Average Ratings,Number of Pages,Published Year,Series,Genres,Awards,Places
0,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows (Harry Po...,J.K. RowlingJ.K. Rowling,29591732959173,136880.0,10.0,1518.0,20072007,2.0,"[Fantasy, Young Adult, Fiction, Fantasy, Young...",[Locus Award Nominee for Best Young Adult Nove...,"[[London, England, (United Kingdom)], [Hogwart..."
1,https://www.goodreads.com/book/show/2767052-th...,"The Hunger Games (The Hunger Games, #1)The Hun...",Suzanne CollinsSuzanne Collins,66847276684727,350980.0,8.0,748.0,20082008,2.0,"[Young Adult, Fiction, Dystopia, Young Adult, ...",[Locus Award Nominee for Best Young Adult Book...,"[[District 12, Panem, Capitol, Panem, Panem, (..."
2,https://www.goodreads.com/book/show/1.Harry_Po...,Harry Potter and the Half-Blood Prince (Harry ...,J.K. RowlingJ.K. Rowling,25750632575063,84024.0,8.0,1304.0,20062006,2.0,"[Fantasy, Young Adult, Fiction, Fantasy, Young...",[Locus Award Nominee for Best Young Adult Nove...,"[[Hogwarts School of Witchcraft and Wizardry, ..."
3,https://www.goodreads.com/book/show/6.Harry_Po...,Harry Potter and the Goblet of Fire (Harry Pot...,J.K. RowlingJ.K. Rowling,27495802749580,99152.0,8.0,1468.0,20022002,2.0,"[Fantasy, Young Adult, Fiction, Fantasy, Young...","[Hugo Award for Best Novel (2001), Mythopoeic ...","[[Hogwarts School of Witchcraft and Wizardry,\..."
4,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix (Har...,J.K. RowlingJ.K. Rowling,26530652653065,90742.0,8.0,1740.0,20042004,2.0,"[Fantasy, Young Adult, Fiction, Fantasy, Young...",[Bram Stoker Award for Works for Young Readers...,"[[Hogwarts School of Witchcraft and Wizardry, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.goodreads.com/book/show/2744.Anans...,Anansi BoysAnansi Boys,Neil GaimanNeil Gaiman,202182202182,20398.0,8.0,774.0,20062006,2.0,"[Fantasy, Fiction, Urban Fantasy, Fantasy, Fic...","[Locus Award for Best Fantasy Novel (2006), My...","[[London, England, (United Kingdom)], [Florida..."
96,https://www.goodreads.com/book/show/16527.Daug...,Daughter of FortuneDaughter of Fortune,Isabel AllendeIsabel Allende,119444119444,8316.0,8.0,864.0,20062006,2.0,"[Historical Fiction, Fiction, Historical, Hist...",[Premio de traducción literaria Valle Inclán N...,"[[California, (United States)], [Valparaíso, (..."
97,https://www.goodreads.com/book/show/4965.Year_...,Year of WondersYear of Wonders,Geraldine BrooksGeraldine Brooks,146297146297,21162.0,8.0,608.0,20022002,0.0,"[Historical Fiction, Fiction, Historical, Hist...","[ALA Alex Award (2002), ALA Alex Award (2002)]","[[Eyam, Derbyshire, England,\n1666, (United Ki..."
98,https://www.goodreads.com/book/show/28186.The_...,The Sea of Monsters (Percy Jackson and the Oly...,Rick RiordanRick Riordan,NaN,NaN,8.0,NaN,NaN,NaN,NaN,[],NaN
